# Strategies and Panel Data

When working with panel data, we have two options.

1. We can apply a strategy to all stocks in the panel data.

1. We can subset a particular stock, effectively obtaining time series data for that single asset. From here we can use the same method we did in the previous notebook. This has the advantage of parameter optimisation, which we'll see at the end of this notebook.

Let's start with the *one strategy for all stocks* approach, applying the SMA crossover strategy.

First, imports and preparing the dataframe.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [4]:
df = pd.read_csv("data/top_six_2020_2025.csv")
df.DlyCalDt = pd.to_datetime(df.DlyCalDt, dayfirst=True)

### Exercise: Recycling Code

Whenever we're going to be repeating an operation, it is usually a good idea (and sometimes necessary) to create our own function. The code below is correct, but out-of-order. Re-arrange the below into a function called `calculate_sma`, which will take three inputs:
* a data frame that contains a prccd column
* a fast window size
* a slow window size

The function will add as features:
* slow and fast moving averages based on the prccd price
* buy/sell signals
* a position

In [5]:
# dfi["FastMA"] = dfi.DlyClose.rolling(window=fast_window).mean()

# dfi["Position"] = dfi.Signal.shift()

# dfi.Signal = np.where(dfi.SlowMA.isna(), 0, dfi.Signal)

# return dfi

# dfi["Signal"] = np.where(dfi.FastMA > dfi.SlowMA, 1, -1)

# dfi["SlowMA"] = dfi.DlyClose.rolling(window=slow_window).mean()

# def calculate_sma(dfi, fast_window, slow_window):

In [9]:
FAST, SLOW = 50, 200
grouped = df.groupby("Ticker")
groups = []
for name, group in grouped:
    print(f"Now processing signals for {name}...")
    signals= calculate_sma(group, FAST, SLOW)
    groups.append(signals)

df = pd. concat(groups)
df


Now processing signals for AAPL...


NameError: name 'calculate_sma' is not defined

With the function done, we can apply it groupwise to the stocks in our panel data by iterating over the groups.

Then it's a matter of calculating the cumulative returns as we've done before.... nearly. Remember that we have grouped data here!

In [11]:
df["Returns"] = df.groupby("Ticker").DlyClose.pct_change()
df["Strategy"] = df.groupby("Ticker").Returns * df.Signal.shift()

df["Buyout"] = (1 + df.Returns).groupby(df.Ticker).cumprod() -1
df["MACS"] = (1 + df.Strategy).groupby(df.Ticker).cumprod() - 1
df.tail()

AttributeError: 'DataFrame' object has no attribute 'Signal'

### Exercise: Data Display

Can you produce a data frame with the end-of-period market and strategy cumulative return for each stock?

In [1]:
df.groupby ("Ticker").agg(
    {"Buyout": "last", "MACS": "last"}
    
)

NameError: name 'df' is not defined

## Single Stock

Let's subset META (Facebook) and use that for the rest of the notebook.

### Exercise: BB Breakout

Implement an Bollinger Bands Breakout strategy and backtest it on META.

- First create high and low Bollinger Bands as usual
- Then generate signals as follows:
  - Buy (+1) when the close price crosses above the upper band
    - Hold the position until a signal change
  - Sell (-1) when the close price crosses under the lower band
    - Hold the position until a signal change
- Generate positions
- Calculate the strategy returns and cumulative strategy returns
- Report and plot the cumulative strategy and market returns

**HINT** You can create an empty column by assigning it `np.nan`

In [ ]:
## YOUR CODE GOES HERE

## Parameter Optimisation

Parameter optimisation involves getting the best out of a strategy. The *parameters* in our strategy are the window size of the fast SMA and slow SMA. We used 50 and 200 days, but could try different combinations to see if they give better results.

Common **fast** window sizes include 7, 20, 50 and common **slow** window sizes include 50, 100, 200.

However, beware of overfitting! When a model fits our historical data too closely, it can perform poorly in future real-world scenarios.

### Exercise: Which windows?

The code below implements an Exponential Moving Average Crossover Strategy. Similar to our SMA strategy, but calculating the rolling mean in a different way that takes recent prices into greater consideration.

Unfortunately, the implementation with the window sizes below does not perform well. Modify the code below to perform a parameter optimisation on the strategy. Your code should be able to test 3 fast window sizes and 3 slow window sizes in combination.

Can you find a combination of window sizes that sees this EMA strategy outperform the simple Bollinger Band strategy above?

In [ ]:
ema = meta.copy()

## MODIFY THIS CODE

fast_window, slow_window = 50, 200

ema["FastEMA"] = ema.DlyClose.ewm(span=fast_window, adjust=False).mean()
ema["SlowEMA"] = ema.DlyClose.ewm(span=slow_window, adjust=False).mean()

ema["Signal"] = np.where(ema.FastEMA > ema.SlowEMA, 1, -1)
ema["Signal"] = np.where(ema.SlowEMA.isna(), 0, ema.Signal)

ema["Position"] = ema.Signal.shift()
ema["StratRet"] = ema.Position * ema.MarketDaily
ema["Strategy"] = (1 + ema.StratRet).cumprod() - 1

print("Short window:", fast_window, "Long window:", slow_window)
print("Cumulative Strategy Return is", ema.Strategy.iloc[-1])